In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Import data

In [0]:
import pandas as pd
import numpy as np
from pprint import pprint
import os
from tqdm import tqdm

In [0]:
path="drive/My Drive/1001_project_Total/Data/"
df=pd.read_csv(path+"20170101.csv")
df=df.drop('Unnamed: 0',1)

In [16]:
df.columns

Index(['profile', 'category1', 'canonicalUrl', 'referrerHostClass',
       'sessionStop', 'userId', 'publishtime', 'sessionStart', 'referrerUrl',
       'keywords', 'id', 'eventId', 'city', 'title', 'url', 'country',
       'region', 'author', 'referrerSearchEngine', 'deviceType', 'time', 'os',
       'activeTime'],
      dtype='object')

In [22]:
len(df["category1
"].unique())

77

# Adding Sessions

In [26]:
t=df.sort_values(["id","time"])

# according from the dataset description: There is the threshold when computing the “activeTime”, when it is too short then it won’t be account in original dataset
t["activeTime"]=t["activeTime"].fillna(0)

# find the sessions with total time more than 0 and less than 45 minutes
t["sessionNo"]=t["sessionStart"].cumsum()
t["activeTotalTime"]=t.groupby('sessionNo')['activeTime'].transform('sum')
print("The number of total events by users is: ",len(t))
print("The number of sessions is: ",max(t["sessionNo"]))
print("The number of unique users is: ",len(t["userId"].unique()))
t["sessionNo"]=t["sessionStart"].cumsum()
t=t[t["activeTotalTime"]>0]
t=t[t["activeTotalTime"]<=2700]

The number of total events by users is:  1513739
The number of sessions is:  518675
The number of unique users is:  248892


In [28]:
print("The number of total events by users is: ",len(t))
print("The number of sessions is: ",len(t["sessionNo"].unique()))
print("The number of unique users is: ",len(t["userId"].unique()))

The number of total events by users is:  1264266
The number of sessions is:  378103
The number of unique users is:  199076


In [29]:
t[["sessionNo","id","time","sessionStart","sessionStop","activeTime","activeTotalTime"]].iloc[0:10]

,sessionNo,id,time,sessionStart,sessionStop,activeTime,activeTotalTime
1014858,2,000dd1bf05d3e8bfcf581f8f95dde7b8f27f5af2,1483291167,True,False,13.0,13.0
1179225,7,0075258a2dfdf69a01411985daed9d7b021672f1,1483299359,True,True,0.0,25.0
1126879,7,0076aaa81a84345caf73bede3ce4e6f5875b56d6,1483296990,False,False,25.0,25.0
1130392,7,0076aaa81a84345caf73bede3ce4e6f5875b56d6,1483297146,False,True,0.0,25.0
1081924,12,00905cfb7b640ff5211758d002d4b66e50a2984c,1483294633,True,True,0.0,21.0
1114717,12,00af22160c2e9148a8731899d978649ff3ccb8f6,1483296382,False,False,21.0,21.0
1301278,13,00bc12362d794e7897ed764ebdc741284beb37bf,1483303999,True,True,0.0,66.0
1437134,13,00bc12362d794e7897ed764ebdc741284beb37bf,1483308471,False,False,66.0,66.0
693175,13,00cd49326858ce3ead4bae2c832d26e571872749,1483275044,False,False,0.0,66.0
693654,13,00cd49326858ce3ead4bae2c832d26e571872749,1483275066,False,False,0.0,66.0


In [11]:
articles=t["id"].unique()
d={}
for i in range(len(articles)):
    article=articles[i]
    d[article]=i
t["articleId"]=t["id"].map(d)
print("The length of t is: ",len(t))
t.head()

The length of t is:  1264266


,profile,category1,canonicalUrl,referrerHostClass,sessionStop,userId,publishtime,sessionStart,referrerUrl,keywords,id,eventId,city,title,url,country,region,author,referrerSearchEngine,deviceType,time,os,activeTime,sessionNo,activeTotalTime,articleId
1014858,"[{'item': '0', 'groups': [{'count': 1, 'group'...",NaN,http://www.adressa.no/forbruker/article1827939...,search,False,cx:2riybj66fbna7d3wuenvszltq:20ihq0a49l3xl,2012-05-24T17:57:46.000Z,True,http://google.no,"nyheter,trondheim,ukeadressa,eadressa,skatteli...",000dd1bf05d3e8bfcf581f8f95dde7b8f27f5af2,393048540,trondheim,Bruk Instagram til å sende oss bildene,http://adressa.no/forbruker/article1827939.ece,no,sor-trondelag,yngve bergli,Google,Mobile,1483291167,iPhone OS,13.0,1,13.0,0
1179225,"[{'item': '0', 'groups': [{'count': 1, 'group'...",NaN,http://www.adressa.no/kultur/article11367883.ece,search,True,cx:1q3fiidlreq0516tc5glqpi4ms:n1qgh59trqqi,2015-07-30T14:15:32.000Z,True,http://google.no,"anmeldelser,det skjer,film,musikk,tv,museum,sc...",0075258a2dfdf69a01411985daed9d7b021672f1,30998606,ski,Her blir Paul-Ottar overrasket på 50-årsdagen,http://adressa.no/kultur/article11367883.ece,no,akershus,geir tønset,Google,Mobile,1483299359,iPhone OS,0.0,2,25.0,1
1126879,"[{'item': '0', 'groups': [{'count': 1, 'group'...",NaN,http://www.adressa.no/vaeret/article10986427.ece,internal,False,cx:2mc1hmvfd2jy3vrksqst72om2:tzu89xjs87e4,NaN,False,http://adressa.no/vaeret/article10997682.ece,NaN,0076aaa81a84345caf73bede3ce4e6f5875b56d6,1136968678,sande,Stengt inne i kjølig luft av jetstrømmen,http://adressa.no/vaeret/article10986427.ece,no,vestfold,NaN,NaN,Tablet,1483296990,iPhone OS,25.0,2,25.0,2
1130392,"[{'item': '0', 'groups': [{'count': 1, 'group'...",NaN,http://www.adressa.no/vaeret/article10986427.ece,internal,True,cx:2mc1hmvfd2jy3vrksqst72om2:tzu89xjs87e4,NaN,False,http://adressa.no/vaeret/article10997682.ece,NaN,0076aaa81a84345caf73bede3ce4e6f5875b56d6,804552954,sande,Stengt inne i kjølig luft av jetstrømmen,http://adressa.no/vaeret/article10986427.ece,no,vestfold,NaN,NaN,Tablet,1483297146,iPhone OS,0.0,2,25.0,2
1081924,"[{'item': '0', 'groups': [{'count': 1, 'group'...",NaN,http://www.adressa.no/kultur/2016/12/15/%c3%98...,search,True,cx:2f26eami94xio2zhm874z8od1f:3qdjlrsq10h9b,2016-12-15T14:20:27.000Z,True,http://google.no/search,"anmeldelser,det skjer,film,musikk,tv,museum,sc...",00905cfb7b640ff5211758d002d4b66e50a2984c,767945697,oslo,Øremerket NRK-avgift skal være teknologinøytral,http://adressa.no/kultur/2016/12/15/%c3%98reme...,no,oslo,ntb,Google,Mobile,1483294633,Android,0.0,3,21.0,3


In [0]:
t.to_csv(path+"20170101_prestage.csv")